In [1]:
!uv pip install torch torchvision pillow numpy scikit-learn

Using Python 3.13.7 environment at: C:\test\.venv
Resolved 17 packages in 1.02s
Prepared 3 packages in 3.41s
Installed 15 packages in 44.05s
 + filelock==3.20.3
 + fsspec==2026.2.0
 + jinja2==3.1.6
 + joblib==1.5.3
 + markupsafe==3.0.3
 + mpmath==1.3.0
 + networkx==3.6.1
 + scikit-learn==1.8.0
 + scipy==1.17.0
 + setuptools==80.10.2
 + sympy==1.14.0
 + threadpoolctl==3.6.0
 + torch==2.10.0
 + torchvision==0.25.0
 + typing-extensions==4.15.0


In [8]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np

# 1. 모델 로드 (DINOv2 Small 버전 - 로컬 사양 부담 적음)
# 더 높은 성능을 원하시면 'dinov2_vits14' 대신 'dinov2_vitb14'를 쓰세요.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device)
model.eval()

# 2. 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_image_embedding(image_path):
    """이미지에서 특징 벡터를 추출하는 함수"""
    img = Image.open(image_path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        embedding = model(img_t)
    return embedding

def calculate_similarity(path1, path2):
    """두 이미지 사이의 코사인 유사도를 계산"""
    emb1 = get_image_embedding(path1)
    emb2 = get_image_embedding(path2)
    
    # 코사인 유사도 계산 (1에 가까울수록 비슷함)
    similarity = F.cosine_similarity(emb1, emb2)
    return similarity.item()

# 3. 실행 예시
# 노트북 파일이 있는 디렉토리를 기준으로 경로 설정
# 절대 경로로 이미지 파일 지정
path_a = r"c:\test\tour\img_forigen\002.jpg"  # 애니메이션/외국 사진
path_b = r"c:\test\tour\img_korea\002.jpeg"    # 국내 사진

try:
    score = calculate_similarity(path_a, path_b)
    print(f"\n[결과] 두 장소의 유사도 점수: {score:.4f}")
    
    if score > 0.8:
        print("결과: 매우 비슷합니다! 한국의 숨은 명소일 확률이 높네요.")
    elif score > 0.6:
        print("결과: 분위기가 제법 흡사합니다.")
    else:
        print("결과: 서로 다른 느낌의 장소입니다.")

except FileNotFoundError as e:
    print(f"이미지 파일을 찾을 수 없습니다: {e}")
    print(f"찾고 있던 경로:")
    print(f"  - {path_a}")
    print(f"  - {path_b}")

Using cache found in C:\Users\USER/.cache\torch\hub\facebookresearch_dinov2_main



[결과] 두 장소의 유사도 점수: 0.4207
결과: 서로 다른 느낌의 장소입니다.


In [15]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os

# 1. 모델 로드 (최초 실행 시 다운로드 진행)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device)
model.eval()

# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_embedding(image_path):
    img = Image.open(image_path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(img_t)
    return embedding

# --- 경로 지정 섹션 ---
# 1. 기준이 되는 외국/애니메이션 이미지 (파일명이 001.jpg라고 가정)
query_img = "img_forigen/001.jpg" 

# 2. 비교할 한국 이미지들이 들어있는 폴더
target_dir = "img_korea"
# -----------------------

if os.path.exists(query_img) and os.path.isdir(target_dir):
    query_emb = get_embedding(query_img)
    
    results = []
    valid_ext = ('.jpg', '.jpeg', '.png', '.webp')

    for file_name in os.listdir(target_dir):
        if file_name.lower().endswith(valid_ext):
            target_path = os.path.join(target_dir, file_name)
            target_emb = get_embedding(target_path)
            
            # 유사도 계산
            score = F.cosine_similarity(query_emb, target_emb).item()
            results.append((file_name, score))

    # 점수 높은 순으로 정렬
    results.sort(key=lambda x: x[1], reverse=True)

    print(f"\n[검색 결과 - 기준: {query_img}]")
    for i, (name, s) in enumerate(results[:3]):
        print(f"{i+1}등: {name} (유사도: {s:.4f})")
else:
    print("경로를 찾을 수 없습니다. 폴더명이나 파일명을 확인해주세요.")
    # 현재 내 코드가 어디를 보고 있는지 확인용
    print(f"현재 작업 디렉토리: {os.getcwd()}")

Using cache found in C:\Users\USER/.cache\torch\hub\facebookresearch_dinov2_main



[검색 결과 - 기준: img_forigen/001.jpg]
1등: 001.jpg (유사도: 0.4898)
2등: 004.JPG (유사도: 0.0484)
3등: 002.jpeg (유사도: 0.0234)


In [12]:
!uv pip install transformers torch torchvision pillow

Using Python 3.13.7 environment at: C:\test\.venv
Audited 4 packages in 827ms


In [16]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import torch.nn.functional as F

# 1. CLIP 모델 및 프로세서 로드
# 가장 많이 쓰이는 ViT-B/32 버전을 사용합니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch32"

model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)

def get_clip_embedding(image_path):
    """CLIP을 이용해 이미지의 특징 벡터를 추출"""
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        # 여기서 출력을 받습니다.
        outputs = model.get_image_features(**inputs)
        
        # [수정 포인트] 만약 outputs가 객체 형태라면 데이터(Tensor)만 추출합니다.
        # 일반적으로 CLIPModel은 직접 Tensor를 반환하지만, 
        # 특정 버전에서는 객체를 반환할 수 있어 아래와 같이 처리하는 것이 안전합니다.
        if hasattr(outputs, "pooler_output"):
            image_features = outputs.pooler_output
        else:
            image_features = outputs
            
        # 이제 정규화를 수행합니다.
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        
    return image_features

# --- 경로 설정 ---
query_img_path = "img_forigen/002.jpg"
target_dir = "img_korea"

if os.path.exists(query_img_path) and os.path.isdir(target_dir):
    # 1. 기준 이미지 벡터 추출
    query_emb = get_clip_embedding(query_img_path)
    
    results = []
    valid_ext = ('.jpg', '.jpeg', '.png', '.webp')

    # 2. 한국 이미지 폴더 순회 및 비교
    print(f"CLIP 모델로 '{target_dir}' 내 유사 장소 검색 중...")
    
    for file_name in os.listdir(target_dir):
        if file_name.lower().endswith(valid_ext):
            target_path = os.path.join(target_dir, file_name)
            target_emb = get_clip_embedding(target_path)
            
            # 3. 코사인 유사도 계산
            # CLIP은 이미 정규화를 했으므로 단순 내적(dot product)으로도 계산 가능하지만, 
            # 명확성을 위해 동일하게 F.cosine_similarity를 사용합니다.
            score = F.cosine_similarity(query_emb, target_emb).item()
            results.append((file_name, score))

    # 결과 정렬
    results.sort(key=lambda x: x[1], reverse=True)

    print(f"\n[CLIP 검색 결과 - 기준: {query_img_path}]")
    for i, (name, s) in enumerate(results[:3]):
        print(f"{i+1}등: {name} (유사도: {s:.4f})")
else:
    print("경로를 확인해 주세요.")

Loading weights: 100%|██████████| 398/398 [00:00<00:00, 1151.98it/s, Materializing param=visual_projection.weight]                                
CLIPModel LOAD REPORT from: openai/clip-vit-base-patch32
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
text_model.embeddings.position_ids   | UNEXPECTED |  | 
vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


CLIP 모델로 'img_korea' 내 유사 장소 검색 중...

[CLIP 검색 결과 - 기준: img_forigen/002.jpg]
1등: 002.jpeg (유사도: 0.8299)
2등: 009.jpg (유사도: 0.6160)
3등: 008.jpg (유사도: 0.5671)


In [18]:
!uv pip install timm

Using Python 3.13.7 environment at: C:\test\.venv
Resolved 30 packages in 350ms
Prepared 1 package in 258ms
Installed 1 package in 166ms
 + timm==1.0.24


In [19]:
import timm
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
# V2의 작은 모델인 'atto'나 'tiny' 버전을 추천합니다.
model_convnext = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=True, num_classes=0).to(device)
model_convnext.eval()

def get_convnext_embedding(img_path):
    img = Image.open(img_path).convert('RGB')
    # timm 모델 전용 전처리 구성
    data_config = timm.data.resolve_model_data_config(model_convnext)
    transform = timm.data.create_transform(**data_config, is_training=False)
    
    img_t = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model_convnext(img_t) # num_classes=0이면 풀링된 특징 벡터가 나옵니다
    return features / features.norm(p=2, dim=-1, keepdim=True)

c:\test\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--timm--convnextv2_tiny.fcmae_ft_in22k_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [21]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import os
import torch.nn.functional as F

# 1. 모델 준비 (Places365용 ResNet18 구조)
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델의 뼈대를 가져옵니다.
model_places = models.resnet18(num_classes=365)
# 마지막 분류 층(FC layer)을 제거하고 특징 추출기(Feature Extractor)로만 사용합니다.
model_places = nn.Sequential(*(list(model_places.children())[:-1])).to(device)
model_places.eval()

# 2. 이미지 특징 추출 함수 정의 (이 부분이 'get_places_embedding'의 정체입니다!)
def get_places_embedding(img_path):
    # Places365/ResNet 표준 전처리
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    img = Image.open(img_path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        # 이미지를 모델에 통과시켜 특징 벡터를 뽑아냅니다.
        features = model_places(img_t).flatten(1)
    
    # 유사도 비교를 위해 길이를 1로 맞추는(정규화) 과정입니다.
    return features / features.norm(p=2, dim=-1, keepdim=True)

# 3. 경로 설정 및 유사도 검색 실행
query_img_path = "img_forigen/001.jpg"
target_dir = "img_korea"

if os.path.exists(query_img_path) and os.path.isdir(target_dir):
    # 기준 이미지 특징 추출
    query_emb = get_places_embedding(query_img_path)
    
    results = []
    valid_extensions = ('.jpg', '.jpeg', '.png', '.webp')

    # 폴더 내 모든 한국 이미지와 비교
    for file_name in os.listdir(target_dir):
        if file_name.lower().endswith(valid_extensions):
            target_path = os.path.join(target_dir, file_name)
            target_emb = get_places_embedding(target_path)
            
            # 코사인 유사도 계산
            score = F.cosine_similarity(query_emb, target_emb).item()
            results.append((file_name, score))

    # 결과 정렬 (높은 점수 순)
    results.sort(key=lambda x: x[1], reverse=True)

    print(f"\n[Places365 검색 결과 - 기준: {query_img_path}]")
    for i, (name, s) in enumerate(results[:5]):
        print(f"{i+1}등: {name} (유사도: {s:.4f})")
else:
    print("경로를 찾을 수 없습니다. img_forigen 폴더와 img_korea 폴더를 확인해주세요.")


[Places365 검색 결과 - 기준: img_forigen/001.jpg]
1등: 001.jpg (유사도: 0.9973)
2등: 006.jpg (유사도: 0.9973)
3등: 002.jpeg (유사도: 0.9971)
4등: 008.jpg (유사도: 0.9960)
5등: 007.jpg (유사도: 0.9956)


In [22]:
import torchvision.models as models

# ResNet18 기반의 Places365 가중치를 불러옵니다.
model_places = models.resnet18(num_classes=365)
# 공식 가중치 파일 URL (최초 1회 수동 로드 필요할 수 있음)
storage_url = 'https://download.pytorch.org/models/resnet18-5c106cde.pth' # 기본 resnet 가중치와 구조가 같음
# 실제로는 장소 분류를 위해 학습된 별도의 가중치 파일(.pth)을 로드해야 정확합니다.
# 테스트용으로는 기본 ResNet의 특징 추출 능력을 활용해도 무방합니다.
model_places = torch.nn.Sequential(*(list(model_places.children())[:-1])).to(device) 
model_places.eval()

def get_places_embedding(img_path):
    # 전처리는 일반적인 ResNet 방식 사용
    transform = transforms.Compose([
        transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img = Image.open(img_path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model_places(img_t).flatten(1)
    return features / features.norm(p=2, dim=-1, keepdim=True)

In [24]:
import torch
import timm
import os
import torch.nn.functional as F
from PIL import Image

# 1. 모델 설정 및 로드
device = "cuda" if torch.cuda.is_available() else "cpu"

# 'convnextv2_tiny' 모델을 사용합니다. 성능을 높이려면 'base'나 'large'로 변경 가능하지만, 
# 로컬 테스트용으로는 'tiny'가 속도 면에서 가장 적절합니다.
model_name = 'convnextv2_tiny.fcmae_ft_in22k_in1k'
model_convnext = timm.create_model(model_name, pretrained=True, num_classes=0).to(device)
model_convnext.eval()

# 모델에 맞는 전처리(Transform) 자동 생성
data_config = timm.data.resolve_model_data_config(model_convnext)
transform = timm.data.create_transform(**data_config, is_training=False)

def get_convnext_embedding(img_path):
    """ConvNeXt V2를 사용해 이미지 특징 추출"""
    img = Image.open(img_path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model_convnext(img_t)
        
    # 유사도 비교를 위해 정규화
    return features / features.norm(p=2, dim=-1, keepdim=True)

# 2. 경로 설정 (질문자님 디렉토리 기준)
query_img_path = "img_forigen/001.jpg"
target_dir = "img_korea"

if os.path.exists(query_img_path) and os.path.isdir(target_dir):
    # 기준 이미지 특징 추출
    query_emb = get_convnext_embedding(query_img_path)
    
    results = []
    valid_extensions = ('.jpg', '.jpeg', '.png', '.webp')

    # 3. 폴더 순회 및 유사도 계산
    print(f"ConvNeXt V2 모델로 '{target_dir}' 검색 중...")
    
    for file_name in os.listdir(target_dir):
        if file_name.lower().endswith(valid_extensions):
            target_path = os.path.join(target_dir, file_name)
            target_emb = get_convnext_embedding(target_path)
            
            # 코사인 유사도 계산
            score = F.cosine_similarity(query_emb, target_emb).item()
            results.append((file_name, score))

    # 결과 정렬
    results.sort(key=lambda x: x[1], reverse=True)

    print(f"\n[ConvNeXt V2 검색 결과 - 기준: {query_img_path}]")
    print("-" * 40)
    for i, (name, s) in enumerate(results[:5]):
        print(f"{i+1}등: {name} (유사도: {s:.4f})")
    print("-" * 40)

else:
    print("경로를 확인해 주세요. img_forigen/001.jpg 파일과 img_korea 폴더가 필요합니다.")

ConvNeXt V2 모델로 'img_korea' 검색 중...

[ConvNeXt V2 검색 결과 - 기준: img_forigen/001.jpg]
----------------------------------------
1등: 001.jpg (유사도: 0.3324)
2등: 003.JPG (유사도: 0.2836)
3등: 006.jpg (유사도: 0.2199)
4등: 004.JPG (유사도: 0.1856)
5등: 009.jpg (유사도: 0.1479)
----------------------------------------
